## Machine Hack Buyer's Time Prediction - Feature Engineering

Buyers spend a significant amount of time surfing an e-commerce store, since the pandemic the e-commerce has seen a boom in the number of users across the domains. In the meantime, the store owners are also planning to attract customers using various algorithms to leverage customer behavior patterns.

Tracking customer activity is also a great way of understanding customer behavior and figuring out what can actually be done to serve them better. Machine learning and AI has already played a significant role in designing various recommendation engines to lure customers by predicting their buying patterns.

We are provided with the visitor's session data and are required to create a regression algorithm to predict the time a buyer will spend on the platform.

### Attribute Description
- `session_id`: Unique identifier for every row
- `session_number`: Session type identifier
- `client_agent`: Client-side software details
- `device_details`: Client-side device details
- `date`: Datestamp of the session
- `purchased`: Binary value for any purchase done
- `added_in_cart`: Binary value for cart activity
- `checked_out`:  Binary value for checking out successfully
- `time_spent`: Total time spent in seconds (Target Column)

Here the analysis will be focused on creating features, particularly (and most importantly) based on `client_agent` feature.

In [4]:
import os
for dirname, _, filenames in os.walk('ParticipantData_BTPC/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

ParticipantData_BTPC/.DS_Store
ParticipantData_BTPC/Sample Submission.csv
ParticipantData_BTPC/Test.csv
ParticipantData_BTPC/Train.csv


### Import Packages

In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd, re, requests
from sklearn import preprocessing

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

### Read data

In [6]:
## Read train and test data

train =  pd.read_csv('ParticipantData_BTPC/Train.csv', low_memory = False ,parse_dates = ['date'])
test = pd.read_csv('ParticipantData_BTPC/Test.csv', low_memory = False ,parse_dates = ['date'])

print(f'Shape of train data: {train.shape}')
print(f'Shape of test data: {test.shape}')

Shape of train data: (5429, 9)
Shape of test data: (2327, 8)


In [7]:
## Create a column with split identifier

train['split_ID'] = 'Train'
test['split_ID'] = 'Test'

all_df = pd.concat([train, test])

In [8]:
print(f'Shape of combined dataframe: {all_df.shape}')
display(all_df.head())

Shape of combined dataframe: (7756, 10)


,session_id,session_number,client_agent,device_details,date,purchased,added_in_cart,checked_out,time_spent,split_ID
0,57f879e70d3c5fc2a98102d64c9fd84e,715,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Desktop - Chrome,2020-01-22,1,0,0,236.886,Train
1,a5442b0c7c33d0a811e7661e556b2de8,55,Product/8.0 iPhone/8.1.3,iPhone - iOS,2020-02-27,1,0,0,1277.455,Train
2,305cb1486ed8610c00b37007926cb2c4,11,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,iPhone - MobileWeb,2019-08-01,0,0,0,21.704,Train
3,f2c1ecc9993f0071df91ba178450498c,2794,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Desktop - IE,2019-12-30,0,1,0,455.201,Train
4,e460830ae295e55d2216ebdc761ab9a6,3674,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like ...,iPhone - Web,2019-09-10,0,0,0,1164.877,Train


### Adding datetime features
Add datetime features such as `year`, `quarter`, `weekofyear`, `month`, `day`, `hour`, `minute`, `dayofyear`, `dayofweek` and `is_season`. Values in `season` are like 1, 2, 3, 4. For e.g. 1 represents Dec, Jan and Feb and 2 is March, April and May. 

Quick reference: `{1:'DJF', 2: 'MAM', 3:'JJA', 4:'SON'}`

In [9]:
all_df['year'] = pd.to_datetime(all_df['date']).dt.year
all_df['quarter'] = pd.to_datetime(all_df['date']).dt.quarter
all_df['weekofyear'] = pd.to_datetime(all_df['date']).dt.weekofyear
all_df['month'] = pd.to_datetime(all_df['date']).dt.month
all_df['day'] = pd.to_datetime(all_df['date']).dt.day
all_df['hour'] = pd.to_datetime(all_df['date']).dt.hour
all_df['minute'] = pd.to_datetime(all_df['date']).dt.minute
all_df['dayofyear'] = pd.to_datetime(all_df['date']).dt.dayofyear
all_df['dayofweek'] = pd.to_datetime(all_df['date']).dt.dayofweek
all_df['is_weekend'] = np.where(all_df['dayofweek'].isin([6, 0]), 1, 0)
all_df['season'] = ((all_df['month'] % 12 + 3) // 3)

### Adding a column to store rows where `client_agent` is null

In [10]:
## Create a new column for cases where `client_agent` is null
all_df['ca_isna'] = np.where(all_df['client_agent'].isnull(), 1, 0)

### Column with count of `client_agent` and `device_details` by date

In [11]:
all_df = all_df.fillna('')
ca_bydate_count = (all_df
                   .fillna('')
                   .groupby(['client_agent', 
                             'device_details',
                             'split_ID', 
                             'date'])
                   .size()
                   .to_frame('ca_bydate_count')
                   .reset_index())

all_df = all_df.merge(ca_bydate_count, 
                      on = ['client_agent',
                            'device_details',
                            'split_ID', 
                            'date'],
                      how = 'left')

display(all_df[['client_agent', 'device_details', 'ca_bydate_count']].drop_duplicates())

,client_agent,device_details,ca_bydate_count
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Desktop - Chrome,1
1,Product/8.0 iPhone/8.1.3,iPhone - iOS,2
2,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,iPhone - MobileWeb,1
3,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Desktop - IE,1
4,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like ...,iPhone - Web,6
...,...,...,...
7722,Mozilla/5.0 (iPad; CPU OS 6_1_3 like Mac OS X)...,iPad - Web,1
7727,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,Desktop - IE,1
7735,Product/2.5.1 iPad/6.1.3,iPad - iOS,1
7742,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like ...,iPhone - Web,1


### Add features based on `client_agent`
1. Clean `client_agent` with `to_repl` dictionary values
2. To achieve the above we will make use of a helper function
3. Create new column for each category of replace we did

In [12]:
## Helper function for replacing values
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

## Helper function to create columns after replacement is done
def get_col(col_type):
    all_df['ca_{}'.format(col_type)] = all_df['ca_updated'].apply(
    lambda x: re.findall('{}//.*?\\\\'.format(col_type), x, flags = re.DOTALL))
    all_df['ca_{}'.format(col_type)] = [' '.join(x) for x in list(all_df['ca_{}'.format(col_type)])]
    all_df['ca_{}'.format(col_type)] = (all_df['ca_{}'.format(col_type)]
                        .str.replace('\\', '')
                        .str.replace('{}//'.format(col_type), '')
                        .str.strip())
    return all_df['ca_{}'.format(col_type)]

In [13]:
to_repl = {'Product/': '\\\Product//',
           'Mozilla/': '\\\Mozilla//',
           'AppleWebKit/': '\\\AppleWebKit//',
           'Chrome/': '\\\Chrome//',
           'Safari/': '\\\Safari//',
           'Mobile/': '\\\Mobile//',
           'iPhone/': '\\\iPhone//',
           'Android/': '\\\Android//',
           'Carrier/': '\\\Carrier//',
           'Type/': '\\\Type//',
           'Device/': '\\\Device//',
           'Gecko/': '\\\Gecko//',
           'iPad/': '\\\iPad//',
           'https://': 'https:/',
           'Typhoeus -': '\\\Typhoeus//',
           'Firefox/': '\\\Firefox//',
           'Opera/': '\\\Opera//',
           'Presto/': '\\\Presto//',
           'CFNetwork/': '\\\CFNetwork//',
           'Darwin/': '\\\Darwin//',
           'CriOS/': '\\\CriOS//',
           'Dalvik/': '\\\Dalvik//',
           'GSA/': '\\\GSA//'}

all_df['ca_updated'] = all_df['client_agent'].apply(lambda x: replace_all(x, to_repl))
all_df['ca_updated'] = all_df['ca_updated'] + '\\\\'
all_df['ca_Product'] = get_col('Product')
all_df['ca_Mozilla'] = get_col('Mozilla')
all_df['ca_AppleWebKit'] = get_col('AppleWebKit')
all_df['ca_Chrome'] = get_col('Chrome')
all_df['ca_Safari'] = get_col('Safari')
all_df['ca_Mobile'] = get_col('Mobile')
all_df['ca_iPhone'] = get_col('iPhone')
all_df['ca_Android'] = get_col('Android')
all_df['ca_Carrier'] = get_col('Carrier')
all_df['ca_Type'] = get_col('Type')
all_df['ca_Device'] = get_col('Device')
all_df['ca_Gecko'] = get_col('Gecko')
all_df['ca_iPad'] = get_col('iPad')
all_df['ca_Typhoeus'] = get_col('Typhoeus')
all_df['ca_Firefox'] = get_col('Firefox')
all_df['ca_Opera'] = get_col('Opera')
all_df['ca_Presto'] = get_col('Presto')
all_df['ca_CFNetwork'] = get_col('CFNetwork')
all_df['ca_Darwin'] = get_col('Darwin')
all_df['ca_CriOS'] = get_col('CriOS')
all_df['ca_Dalvik'] = get_col('Dalvik')
all_df['ca_GSA'] = get_col('GSA')
all_df['ca_Chrome'] = all_df['ca_Chrome'].str.replace('Mobile', '')

In [14]:
## Dropping `ca_updated` column
all_df.drop(['ca_updated'], axis = 1, inplace = True)

## Value counts of parsed columns
new_cols = list(all_df.select_dtypes(include = 'O').filter(like = 'ca_', axis = 1).columns)
# new_cols = new_cols[3:]
for col in new_cols:
    print('Value counts in {}:'.format(col)) 
    display(pd.DataFrame([all_df[col].value_counts(ascending = False).head(20), 
                          all_df[col].value_counts(normalize = True, ascending = False).head(20)],
                         index = ['VC', 'Norm_VC']).T)

Value counts in ca_Product:


,VC,Norm_VC
,4817.0,0.621068
4.7.0,387.0,0.049897
4.2.0,284.0,0.036617
4.1.1,205.0,0.026431
4.0.0,175.0,0.022563
6.0,174.0,0.022434
4.6.0,168.0,0.021661
4.4.1,154.0,0.019856
4.2.2,138.0,0.017793
4.5.1,111.0,0.014312


Value counts in ca_Mozilla:


,VC,Norm_VC
,3221.0,0.415291
5.0 (Windows NT 6.1; WOW64),419.0,0.054023
5.0 (iPhone; CPU iPhone OS 7_1_1 like Mac OS X),337.0,0.043450
5.0 (Windows NT 6.3; WOW64; Trident/7.0; Touch; rv:11.0) like Gecko,241.0,0.031073
5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko,225.0,0.029010
5.0 (iPhone; CPU iPhone OS 8_1 like Mac OS X),201.0,0.025915
5.0 (Macintosh; Intel Mac OS X 10_9_5),183.0,0.023595
5.0 (Macintosh; Intel Mac OS X 10_6_8),182.0,0.023466
5.0 (iPhone; CPU iPhone OS 8_1_2 like Mac OS X),159.0,0.020500
5.0 (Macintosh; Intel Mac OS X 10_9_4),124.0,0.015988


Value counts in ca_AppleWebKit:


...

,VC,Norm_VC
,7745.0,0.998582
40.0.2214.69,5.0,0.000645
36.0.1985.49,2.0,0.000258
39.0.2171.50,2.0,0.000258
37.0.2062.52,1.0,0.000129
38.0.2125.67,1.0,0.000129


Value counts in ca_Dalvik:


,VC,Norm_VC
,7755.0,0.999871
2.1.0 (Linux; U; Android 5.0; LG-D855 Build/LRX21R.A1422018487),1.0,0.000129


Value counts in ca_GSA:


,VC,Norm_VC
,7720.0,0.995358
5.2.43972,11.0,0.001418
4.2.2.38484,9.0,0.001160
5.1.42378,7.0,0.000903
3.2.1.25875,5.0,0.000645
4.2.1.37597,2.0,0.000258
4.1.0.31802,2.0,0.000258


### Further cleaning those `ca_*` columns added
1. `ca_Product`: Replacing *'.'* and get first element, replace NAs with UNK
2. `ca_AppleWebKit`: Split by *'/'* and get first element and then replace *' (KHTML, like Gecko)'*, *'(KHTML, like Gecko)'* and *'version'* with blanks. Split by *'.'* and get first element, replace NAs with UNK. Here NAs were converted to *'nan'* when we used `.astype(str)` method.
3. `ca_Mozilla`: Most of the rows have a pattern and use those to clean this column further.
4. `ca_Chrome` and `ca_Safari`: Take only first two part of the versions captured in these columns.
5. `ca_Mobile` and `ca_iPhone`: These again have a pattern i.e. one is available when other is not and yes there are rows where both of these are nulls as well. Added a column to capture that.




In [15]:
## ca_Product
all_df['ca_Product'] = all_df['ca_Product'].str.split('.').str[0].fillna('UNK')
print('Value counts in ca_Product')
display(all_df['ca_Product'].value_counts(dropna = False))

Value counts in ca_Product


          4817
4         1769
3          194
6          174
5           88
9           77
8           62
7           53
150264      38
150288      34
150261      30
160346      27
160362      25
160402      23
160441      22
160432      21
160379      19
160393      18
160406      18
245         18
252         17
160320      16
244         15
160326      13
160418      13
150287      12
160355      11
160318      11
160333      11
150302      10
160314      10
160375      10
160338      10
160309       9
150297       9
160391       7
160447       7
150282       6
160341       5
160330       5
160382       4
160399       3
160387       3
10           3
160360       3
214          2
150259       2
319          1
2            1
Name: ca_Product, dtype: int64

In [16]:
## ca_AppleWebKit
all_df['ca_AppleWebKit'] = (all_df['ca_AppleWebKit'].astype(str)
                            .str.split('/')
                            .str[0]
                            .str.replace(' (KHTML, like Gecko)','', regex = False)
                            .str.replace('(KHTML, like Gecko)', '', regex = False)
                            .str.lower().str.replace('version', '')
                            .str.strip()
                            .str.split('.')
                            .str[0]
                            .str.replace('nan', 'UNK'))
print('Value counts in ca_AppleWebKit')
display(all_df['ca_AppleWebKit'].value_counts(dropna = False))

Value counts in ca_AppleWebKit


       4287
537    2360
600     860
534     168
536      74
533       5
535       2
Name: ca_AppleWebKit, dtype: int64

In [17]:
# ca_Mozilla
suff = []
for row in list(all_df['ca_Mozilla'].fillna('')):
    if (('5.0' in row.lower()) and ('windows nt' in row.lower())):
        suff.append('5.0 Windows NT')
    elif (('5.0' in row.lower()) and ('iphone' in row.lower())):
        suff.append('5.0 iPhone')
    elif (('5.0' in row.lower()) and ('mac os' in row.lower())):
        suff.append('5.0 Mac OS')
    else:
        suff.append('UNK')
all_df['ca_Mozilla'] = pd.Series(suff)

print('Value counts in ca_Mozilla')
display(all_df['ca_Mozilla'].value_counts(dropna = False))

Value counts in ca_Mozilla


UNK               3471
5.0 Windows NT    1617
5.0 Mac OS        1431
5.0 iPhone        1237
Name: ca_Mozilla, dtype: int64

In [18]:
# ca_Chrome and ca_Safari
all_df['ca_Chrome'] = all_df['ca_Chrome'].str[:2].str.strip().fillna('UNK')
all_df['ca_Safari'] = all_df['ca_Safari'].str.split('.').str[0].fillna('UNK')

print('Value counts in ca_Chrome and ca_Safari')
display(all_df['ca_Chrome'].value_counts(dropna = False), all_df['ca_Safari'].value_counts(dropna = False))

Value counts in ca_Chrome and ca_Safari


      6416
39     289
38     236
37     219
40     170
35     138
36     118
41      74
26      36
32      19
30      14
28      11
33       5
34       4
42       2
29       2
18       2
27       1
Name: ca_Chrome, dtype: int64

        5106
537     1553
9537     452
600      423
534      150
8536      26
536       20
533       13
7534      10
535        2
6533       1
Name: ca_Safari, dtype: int64

In [19]:
## ca_Mobile and ca_iPhone
all_df.loc[(all_df['ca_Mobile'] != '') & (all_df['ca_Mobile'].str.len() < 6), 'ca_Mobile'] = all_df['ca_Mobile'].str.zfill(6)
all_df['ca_Mobile'] = all_df['ca_Mobile'].str[:6].str.strip()
all_df.loc[all_df['ca_iPhone'] != '', 'ca_iPhone'] = all_df['ca_iPhone'].str[:1].str.strip()

all_df['ca_Mobile_iPhone'] = np.NaN
all_df.loc[(all_df['ca_iPhone'] != '') & (all_df['ca_Mobile'] == ''), 'ca_Mobile_iPhone'] = all_df['ca_iPhone']
all_df.loc[(all_df['ca_iPhone'] == '') & (all_df['ca_Mobile'] != ''), 'ca_Mobile_iPhone'] = all_df['ca_Mobile']
all_df['ca_Mobile_iPhone'] = all_df['ca_Mobile_iPhone'].fillna('UNK')
del all_df['ca_Mobile'], all_df['ca_iPhone']

print('Value counts in ca_Mobile_iPhone column created')
display(all_df['ca_Mobile_iPhone'].value_counts(dropna = False))

Value counts in ca_Mobile_iPhone column created


UNK       3840
8         1181
7         1040
11D201     372
12B411     201
11D257     186
12B440     182
12A405     135
11A501      86
12B436      67
11B554      65
12B466      63
11D167      56
6           42
11B651      39
12B435      38
11B511      29
10B350      22
10B329      21
12A365      20
12D508      16
12B410      12
11A465       9
09B176       7
11A466       7
11D169       5
10A523       4
5            3
09B206       3
11B601       3
10B141       1
10B146       1
Name: ca_Mobile_iPhone, dtype: int64

In [20]:
display(all_df.head())

,session_id,session_number,client_agent,device_details,date,purchased,added_in_cart,checked_out,time_spent,split_ID,year,quarter,weekofyear,month,day,hour,minute,dayofyear,dayofweek,is_weekend,season,ca_isna,ca_bydate_count,ca_Product,ca_Mozilla,ca_AppleWebKit,ca_Chrome,ca_Safari,ca_Android,ca_Carrier,ca_Type,ca_Device,ca_Gecko,ca_iPad,ca_Typhoeus,ca_Firefox,ca_Opera,ca_Presto,ca_CFNetwork,ca_Darwin,ca_CriOS,ca_Dalvik,ca_GSA,ca_Mobile_iPhone
0,57f879e70d3c5fc2a98102d64c9fd84e,715,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Desktop - Chrome,2020-01-22,1,0,0,236.886,Train,2020,1,4,1,22,0,0,22,2,0,1,0,1,,5.0 Windows NT,537,40,537,,,,,,,,,,,,,,,,UNK
1,a5442b0c7c33d0a811e7661e556b2de8,55,Product/8.0 iPhone/8.1.3,iPhone - iOS,2020-02-27,1,0,0,1277.455,Train,2020,1,9,2,27,0,0,58,3,0,1,0,2,8,UNK,,,,,,,,,,,,,,,,,,,8
2,305cb1486ed8610c00b37007926cb2c4,11,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,iPhone - MobileWeb,2019-08-01,0,0,0,21.704,Train,2019,3,31,8,1,0,0,213,3,0,3,0,1,,5.0 iPhone,537,,,,,,,,,,,,,,,,,,11D257
3,f2c1ecc9993f0071df91ba178450498c,2794,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Desktop - IE,2019-12-30,0,1,0,455.201,Train,2019,4,1,12,30,0,0,364,0,1,1,0,1,,5.0 Windows NT,,,,,,,,,,,,,,,,,,,UNK
4,e460830ae295e55d2216ebdc761ab9a6,3674,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like ...,iPhone - Web,2019-09-10,0,0,0,1164.877,Train,2019,3,37,9,10,0,0,253,1,0,4,0,6,,5.0 iPhone,537,,,,,,,,,,,,,,,,,,11D201


### Label Encoding
Columns label encoded
- `ca_Product`, 
- `ca_Mozilla` 
- `ca_AppleWebKit` 
- `ca_Chrome` 
- `ca_Safari` 
- `ca_Android` 
- `ca_Carrier` 
- `ca_Type` 
- `ca_Device` 
- `ca_Gecko` 
- `ca_iPad` 
- `ca_Typhoeus` 
- `ca_Firefox` 
- `ca_Opera` 
- `ca_Presto` 
- `ca_CFNetwork` 
- `ca_Darwin` 
- `ca_CriOS` 
- `ca_Dalvik` 
- `ca_GSA` 
- `ca_Mobile_iPhone` 
- `device_details`

We would be also dropping these columns from all_df dataframe

In [21]:
le_cols = list(all_df.select_dtypes(include = 'O').filter(regex = 'ca_', axis = 1).columns)
le_cols.extend(['device_details'])

for col in le_cols:
    print(f'Label Encoded Col: {col}')
    LE = preprocessing.LabelEncoder()
    all_df[col + '_enc_le'] = LE.fit_transform(all_df[col].astype(str))
    display(pd.DataFrame([all_df[col], all_df[col + '_enc_le']], index = ['Values', 'LE Values']).T.drop_duplicates())
    all_df.drop(col, axis = 1, inplace = True)

Label Encoded Col: ca_Product


,Values,LE Values
0,,0
1,8,47
5,160402,30
8,4,43
18,7,46
23,160441,34
28,3,41
33,6,45
50,160338,17
82,160333,16


Label Encoded Col: ca_Mozilla


,Values,LE Values
0,5.0 Windows NT,1
1,UNK,3
2,5.0 iPhone,2
13,5.0 Mac OS,0


Label Encoded Col: ca_AppleWebKit


...

,Values,LE Values
0,,0
460,5.2.43972,6
827,3.2.1.25875,1
1066,5.1.42378,5
1339,4.2.2.38484,4
2682,4.2.1.37597,3
6019,4.1.0.31802,2


Label Encoded Col: ca_Mobile_iPhone


,Values,LE Values
0,UNK,31
1,8,30
2,11D257,17
4,11D201,16
17,7,29
21,12B411,21
24,12A405,19
34,11D167,14
44,12B436,23
48,11B651,13


Label Encoded Col: device_details


,Values,LE Values
0,Desktop - Chrome,6
1,iPhone - iOS,16
2,iPhone - MobileWeb,14
3,Desktop - IE,8
4,iPhone - Web,15
5,Android Phone - Android,0
6,Desktop - Firefox,7
7,Android Tablet - Web,5
13,iPad - Web,12
19,Other - Other,10


 ### Get a view of dataframe after feature engineering

In [22]:
print(f'Shape of combined dataframe after feature engineering: {all_df.shape}')
display(all_df.head())

Shape of combined dataframe after feature engineering: (7756, 44)


,session_id,session_number,client_agent,date,purchased,added_in_cart,checked_out,time_spent,split_ID,year,quarter,weekofyear,month,day,hour,minute,dayofyear,dayofweek,is_weekend,season,ca_isna,ca_bydate_count,ca_Product_enc_le,ca_Mozilla_enc_le,ca_AppleWebKit_enc_le,ca_Chrome_enc_le,ca_Safari_enc_le,ca_Android_enc_le,ca_Carrier_enc_le,ca_Type_enc_le,ca_Device_enc_le,ca_Gecko_enc_le,ca_iPad_enc_le,ca_Typhoeus_enc_le,ca_Firefox_enc_le,ca_Opera_enc_le,ca_Presto_enc_le,ca_CFNetwork_enc_le,ca_Darwin_enc_le,ca_CriOS_enc_le,ca_Dalvik_enc_le,ca_GSA_enc_le,ca_Mobile_iPhone_enc_le,device_details_enc_le
0,57f879e70d3c5fc2a98102d64c9fd84e,715,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,2020-01-22,1,0,0,236.886,Train,2020,1,4,1,22,0,0,22,2,0,1,0,1,0,1,5,15,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,6
1,a5442b0c7c33d0a811e7661e556b2de8,55,Product/8.0 iPhone/8.1.3,2020-02-27,1,0,0,1277.455,Train,2020,1,9,2,27,0,0,58,3,0,1,0,2,47,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,16
2,305cb1486ed8610c00b37007926cb2c4,11,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,2019-08-01,0,0,0,21.704,Train,2019,3,31,8,1,0,0,213,3,0,3,0,1,0,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,14
3,f2c1ecc9993f0071df91ba178450498c,2794,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,2019-12-30,0,1,0,455.201,Train,2019,4,1,12,30,0,0,364,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,8
4,e460830ae295e55d2216ebdc761ab9a6,3674,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like ...,2019-09-10,0,0,0,1164.877,Train,2019,3,37,9,10,0,0,253,1,0,4,0,6,0,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,15


### Here's a quick recap
We added:
- Datetime features
- Column to store rows where client_agent is null
- Column with count of client_agent and device_details by date
- Features based on client_agent
And done:
- Label encode string columns

Please UPVOTE 👍